In [1]:
import torch
from torchprofile import profile_macs
import re
import os
from tqdm import tqdm

import warnings
        
# Ignore all DeprecationWarnings
warnings.filterwarnings("ignore")

os.sys.path.append("/expanse/lustre/projects/ddp464/mhnguyen/data")

from PIL import Image, ImageDraw
from IPython.display import display

import requests
from PIL import Image
import torch
print(torch.__version__)

from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

model_dict = { "ov_chat" :"llava-hf/llava-onevision-qwen2-7b-ov-chat-hf",
"ov" : "llava-hf/llava-onevision-qwen2-7b-ov-hf",
"si" : "llava-hf/llava-onevision-qwen2-7b-si-hf"
}

# =========== select the model =========

model_name = list(model_dict.keys())[0]

# ========================================

model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_dict[model_name], 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(device = 'cuda')

processor = AutoProcessor.from_pretrained(model_dict[model_name])

def draw_grid_on_image(image, grid_size=(3, 3)):
    """
    Draws a grid on the image.
    
    :param image: PIL Image object
    :param grid_size: Tuple (rows, cols) indicating the grid dimensions
    :return: PIL Image object with grid lines drawn
    """
    draw = ImageDraw.Draw(image)
    width, height = image.size
    grid_width = width // grid_size[1]
    grid_height = height // grid_size[0]
    
    # Draw vertical lines
    for i in range(1, grid_size[1]):
        x = i * grid_width
        draw.line((x, 0, x, height), fill="red", width=2)
    
    # Draw horizontal lines
    for i in range(1, grid_size[0]):
        y = i * grid_height
        draw.line((0, y, width, y), fill="red", width=2)
    
    return image

def parse_result(output_text):
    """
    Parses the model's output to extract "Yes" or "No".
    
    :param output_text: The model's output text
    :return: "Yes" or "No" based on the output
    """
    # Use regex to find "Yes" or "No" in the output
    match = re.search(r'\b(Yes|No)\b', output_text, re.IGNORECASE)
    if match:
        return match.group(1)
    return "No"  # Default to "No" if no match is found

def parse_assistant_response(output_text):
    """
    Parses the model's output to extract only the assistant's response.
    
    :param output_text: The model's output text
    :return: The assistant's response (starting from "Result: Yes" or "Result: No")
    """
    # Find the start of the assistant's response
    assistant_start = output_text.find("assistant")
    if assistant_start == -1:
        return output_text  # If "assistant" is not found, return the entire output
    
    # Extract the assistant's response
    assistant_response = output_text[assistant_start:].split("assistant")[-1].strip()
    return assistant_response

with open('train_fires_final.txt', 'r') as file:
    train_fire_names = [line.strip() for line in file if line.strip()]

# print(fire_names)

2.3.0a0+40ec155e58.nv24.03


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 19.18it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 0 has a total capacity of 31.74 GiB of which 93.38 MiB is free. Process 1524547 has 18.21 GiB memory in use. Including non-PyTorch memory, this process has 13.43 GiB memory in use. Of the allocated memory 13.00 GiB is allocated by PyTorch, and 142.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
palisades_image_dict = {
    0: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274241_%2B00000.jpg",
    1: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274301_%2B00060.jpg",
    2: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274362_%2B00121.jpg",
    3: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274422_%2B00181.jpg",
    6: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274601_%2B00360.jpg",
    12: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736274961_%2B00720.jpg",
    24: "http://legacy-www.hpwren.ucsd.edu/FIgLib/HPWREN-FIgLib-Data/20250107_PalisadesFire_69bravo-e-mobo-c/1736275681_%2B01440.jpg",   
}

In [ ]:
# Load example images (adjust paths to your few-shot images)
example_image_1 = Image.open(requests.get(palisades_image_dict[24], stream=True).raw)  # Image with smoke
example_image_2 = Image.open(requests.get(palisades_image_dict[0], stream=True).raw)  # Image without smoke

# Define few-shot examples (with actual images)
few_shot_examples = [
    # Example 1: Smoke present
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Is there a smoke plume in this image?"},
            {"type": "image", "image": example_image_1}  # Embed image
        ]
    },
    {
        "role": "assistant",
        "content": "Result: Yes. Smoke detected in Row 2, Column 3. Confidence: 9/10."
    },
    
    # Example 2: No smoke
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Is there smoke in this image?"},
            {"type": "image", "image": example_image_2}  # Embed image
        ]
    },
    {
        "role": "assistant",
        "content": "Result: No. No visible smoke plume. Confidence: 9/10."
    }
]

# Current query (your actual input image)
current_image = Image.open(requests.get(palisades_image_dict[12], stream=True).raw)  # Replace with your image
current_query = {
    "role": "user",
    "content": [
        {"type": "text", "text": f"""
        Is there a smoke plume in this image?
        Answer strictly with "Result: Yes" or "Result: No" first.
        If yes, specify grid location (Row X, Column Y).
        Confidence (0-10).
        """},
        {"type": "image", "image": current_image}  # Your input image
    ]
}

# Combine few-shot examples + current query
conversation = few_shot_examples + [current_query]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

In [2]:
current_query

NameError: name 'current_query' is not defined

In [4]:
# Process all images and text
inputs = processor(
    text=processor.apply_chat_template(conversation, add_generation_prompt=True),
    images=[item["content"][1]["image"] for item in conversation if item["role"] == "user"],  # Extract all images
    return_tensors="pt",
    padding=True
).to("cuda", torch.float32)
    
# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=False,
    pad_token_id=model.config.eos_token_id
)

# Decode output
response = processor.decode(output[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


user 
Is there a smoke plume in this image?assistant user 
Is there smoke in this image?assistant user 

        Is there a smoke plume in this image?
        Answer strictly with "Result: Yes" or "Result: No" first.
        If yes, specify grid location (Row X, Column Y).
        Confidence (0-10).
        assistant
Result: Yes
